In [3]:
import os
import pandas as pd
import tensorflow as tf

In [6]:
data_dir = "../../data/dataset/"
subjects = os.listdir(data_dir)
gestures = ["One", "Two"]

In [35]:
dataset = pd.DataFrame()

for subject in subjects:
    for gesture in gestures:
        gesture_dir = os.path.join(data_dir, subject, gesture)
        recordings = os.listdir(gesture_dir)
        for recording in recordings:
            file_path = os.path.join(gesture_dir, recording)
            data = pd.read_csv(file_path)
            data["label"] = gestures.index(gesture)
            dataset = pd.concat([dataset, data])

dataset.reset_index(inplace=True)
dataset

,index,time,rpx,rpy,rpz,lpx,lpy,lpz,rf0x,rf0y,...,lf2x,lf2y,lf2z,lf3x,lf3y,lf3z,lf4x,lf4y,lf4z,label
0,0,0,0.00000,0.000,0.0000,0.0000,0.0,0.0000,0.00000,0.000,...,0.0000,0.000,0.000,0.0000,0.000,0.000,0.0000,0.000,0.0000,0
1,1,1653183626823804,16.75410,193.575,27.2400,-52.4795,173.2,-19.0587,-60.81850,184.615,...,-13.5538,162.081,-113.331,-44.9268,140.322,-103.007,-78.5855,125.848,-74.1292,0
2,2,1653183626832495,16.55130,194.274,27.3420,-52.4795,173.2,-19.0587,-60.36790,183.243,...,-13.5538,162.081,-113.331,-44.9268,140.322,-103.007,-78.5855,125.848,-74.1292,0
3,3,1653183626841145,16.45830,194.612,27.6364,-52.4795,173.2,-19.0587,-59.38080,181.303,...,-13.5538,162.081,-113.331,-44.9268,140.322,-103.007,-78.5855,125.848,-74.1292,0
4,4,1653183626849854,16.02300,194.723,29.0059,-52.4795,173.2,-19.0587,-58.14620,179.490,...,-13.5538,162.081,-113.331,-44.9268,140.322,-103.007,-78.5855,125.848,-74.1292,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5679,563,1653183729996603,6.40800,194.475,49.2916,-52.4795,173.2,-19.0587,5.04201,133.544,...,-13.5538,162.081,-113.331,-44.9268,140.322,-103.007,-78.5855,125.848,-74.1292,1
5680,564,1653183730005308,8.20202,194.450,49.1359,-52.4795,173.2,-19.0587,6.90280,133.468,...,-13.5538,162.081,-113.331,-44.9268,140.322,-103.007,-78.5855,125.848,-74.1292,1
5681,565,1653183730013984,9.92411,194.559,49.0430,-52.4795,173.2,-19.0587,8.64463,133.482,...,-13.5538,162.081,-113.331,-44.9268,140.322,-103.007,-78.5855,125.848,-74.1292,1
5682,566,1653183730022714,11.91960,194.166,49.0959,-52.4795,173.2,-19.0587,10.86780,133.098,...,-13.5538,162.081,-113.331,-44.9268,140.322,-103.007,-78.5855,125.848,-74.1292,1


In [36]:
features = pd.DataFrame()

In [37]:
features["drf1"] = ((dataset["rf1x"] - dataset["rpx"]).pow(2) + \
                    (dataset["rf1y"] - dataset["rpy"]).pow(2) + \
                    (dataset["rf1z"] - dataset["rpz"]).pow(2)).pow(0.5)


In [38]:
features["drf2"] = ((dataset["rf2x"] - dataset["rpx"]).pow(2) + \
                    (dataset["rf2y"] - dataset["rpy"]).pow(2) + \
                    (dataset["rf2z"] - dataset["rpz"]).pow(2)).pow(0.5)

In [39]:
features["label"] = dataset["label"]
features

,drf1,drf2,label
0,0.000000,0.000000,0
1,107.115244,112.392319,0
2,107.101398,112.039226,0
3,107.071644,111.344272,0
4,107.047702,110.358000,0
...,...,...,...
5679,107.848594,109.878866,1
5680,107.865040,109.902742,1
5681,107.883223,109.928295,1
5682,107.897458,109.951179,1


In [45]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [46]:
X = features.drop(columns=["label"]).to_numpy()
y = features["label"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [47]:
clf = KNeighborsClassifier(n_neighbors=7)
clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [49]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.99      0.99      0.99       911

    accuracy                           0.99      1876
   macro avg       0.99      0.99      0.99      1876
weighted avg       0.99      0.99      0.99      1876



In [50]:
import joblib
joblib.dump(clf, "../../model/knn-clf.joblib")

['../../model/knn-clf.joblib']